# Titanic - Machine Learning from Disaster


Kaggle link: https://www.kaggle.com/c/titanic

Link ex2: 
https://www.kaggle.com/code/alessandromajumba/logistic-regression-castelli

https://wandb.ai/ales-2000-09/titanic_kaggle

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import all the needed library and init Weights and Biases

In [ ]:
import numpy as np

import torch
import torch.nn as nn
torch.manual_seed(0)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import pandas as pd

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Label")

import wandb
wandb.login(key=secret_value_0)
wandb.init(project='titanic_kaggle')


We first need to read the datasets

In [ ]:
titanic_training_data = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic_test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
titanic_training_data.head()

Dataframe needs to be cleaned, knowing if some informations are unknown can be very important to determine if someone survived

In [ ]:
def clean_titanic(df, train=True):
    df["Cabin"] = df["Cabin"].apply(lambda x: pd.isna(x)).astype(bool)
    df["Embarked"] = df["Embarked"].apply(lambda x: pd.isna(x)).astype(bool)
    df["AgeNan"] = df["Age"].apply(lambda x: pd.isna(x)).astype(bool)
    df = pd.concat([df, pd.get_dummies(df['Sex'], dtype='bool', prefix='sex_'), pd.get_dummies(df['Pclass'], dtype='bool', prefix='pclass_')], axis=1)
    df = df.drop(['PassengerId', 'Name','Ticket','Sex','Pclass'], axis=1)
    if train:
        df = df.drop(['Survived'], axis=1)
    numeric_features = df.dtypes[(df.dtypes != 'object') & (df.dtypes != 'bool')].index
    df[numeric_features] = df[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
    df["Age"] = df["Age"].fillna(df["Age"].mean())
    df["Fare"] = df["Fare"].fillna(df["Fare"].mean())
    return df

labels = torch.tensor(titanic_training_data["Survived"].values, dtype=torch.float32)
titanic_training_data = clean_titanic(titanic_training_data)
titanic_training_data.head()

We then transform the data from numpy (pandas representation) into torch's `Tensor`

In [ ]:
titanic_data_tensor = torch.tensor(titanic_training_data.astype('float').values, dtype=torch.float32)
titanic_data_tensor.shape

Create a `TensorDataset` to get tuple of data and label

In [ ]:
dataset = torch.utils.data.TensorDataset(titanic_data_tensor, labels)

We then split between the training and validation set

In [ ]:
training_size = int(0.7 * len(dataset))
validation_size = len(dataset) - training_size
train, val = torch.utils.data.random_split(dataset, [training_size, validation_size], generator=torch.Generator().manual_seed(0))
data_loader_train = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
data_loader_val = torch.utils.data.DataLoader(val, batch_size=64, shuffle=True)

Layer initialization using Xavier Uniform on the weight and a constant 0 value on the bias

Create the LinearModel with one Linear layer and Sigmoid applied to the output

In [ ]:
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        # Define a linear layer
        self.linear = nn.Linear(input_size, output_size)
        # Initialize weights with Xavier Uniform
        nn.init.xavier_uniform_(self.linear.weight)
        # Set bias to 0
        nn.init.constant_(self.linear.bias, 0)
    
    def forward(self, x):
        # Apply the linear transformation
        x = self.linear(x)
        # Apply the sigmoid activation function
        x = torch.sigmoid(x)
        return x

Initialize the network (call it `net`, it would makes things easier later), the loss, the optimizer and write the training loop

Don't forget to check the validation loss and save your model at the end of each epoch!

In [ ]:
from torch.autograd import Variable

num_epochs = 400 # should be more than enought, but can be changed
lr = 3e-3 # e.q to 0.003, you can change it if needed

input_size = titanic_training_data.shape[1]
output_size = 1
net = LinearModel(input_size, output_size)

criterion = nn.BCELoss() # I use Binary Cross Entropy Loss as the loss function

optimizer = torch.optim.RMSprop(net.parameters(), lr = lr)

for epoch in range(num_epochs):
    training_loss = 0
    #TRAINING LOOP
    for batch in data_loader_train:
        X = Variable(torch.FloatTensor(batch[0]))
        y = Variable(torch.FloatTensor(batch[1]))
        
        optimizer.zero_grad()
        outputs = net(X).reshape(X.shape[0]).float()
        loss = criterion(outputs, y)
        training_loss += loss
        loss.sum().backward()
        optimizer.step()
    validation_loss = 0
    #VALIDATION LOOP
    with torch.no_grad():
        for batch in data_loader_val:
            X = batch[0]
            y = batch[1]
            
            
            predictions = net(X).reshape(X.shape[0]).float()
            loss = criterion(predictions, y)
            validation_loss += loss
            
    wandb.log({'training_loss': (training_loss/64).item(), 'validation_loss': (validation_loss/64).item()})
    wandb.log({'Net': net.state_dict()})
    print({'epoch':(epoch), 'training_loss': (training_loss/64).item(), 'validation_loss': (validation_loss/64).item()})
    # SAVE THE MODEL
    PATH = "./mymodel.pth"
    torch.save(net.state_dict(), PATH)

This loop computes the prediction on the test dataset and create a submission file

You then just have to click the submit button to get your score, lucky you!

In [ ]:
titanic_test_data_cleaned = clean_titanic(titanic_test_data, train=False)
titanic_data_tensor = torch.tensor(titanic_test_data_cleaned.astype('float').values, dtype=torch.float32)

with torch.no_grad():
    net.eval()
    test_pred = torch.LongTensor()
    for i, data in enumerate(titanic_data_tensor):
        output = net(data)
        predicted = torch.ge(output, 0.5)
        test_pred = torch.cat((test_pred, predicted), dim=0)
    out_df = pd.DataFrame(np.c_[titanic_test_data['PassengerId'].values, test_pred.numpy()], columns=['PassengerId', 'Survived'])
    out_df.to_csv('submission.csv', index=False)